In [ ]:
# Multi-Agent Blog Post Generator using Ollama:
# 1. Research latest news about a topic (DuckDuckGo)
# 2. Write a short blog post in English using Ollama
# 3. Translate it into Portuguese and French using Ollama
# Flow: Start -> Researcher -> Writer -> Translator -> End

# run in terminal: ollama serve 
# run in terminal: ollama pull llama3

from typing import TypedDict, List
from langgraph.graph import StateGraph, END
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate

In [3]:
class AgentState(TypedDict):
    topic: str
    research_data: List[str]  
    blog_post: str            

def researcher_node(state: AgentState):
    topic = state["topic"]
    print(f"Researcher is looking for latest news about: {topic}")
    
    search = DuckDuckGoSearchRun()
    
    try:
        results = search.run(f"latest news and facts about {topic}")
    except Exception as e:
        results = f"Could not find data: {e}"
        
    print("Research complete.")
    
    return {"research_data": state.get("research_data", []) + [results]}

def writer_node(state: AgentState):
    print("Writer is writing the draft...")
    
    topic = state["topic"]
    data = state["research_data"][-1] if state["research_data"] else ""
    
    llm = ChatOllama(model="llama3", temperature=0.7)
    
    prompt = ChatPromptTemplate.from_template(
        """You are a blog writer. 
           Write a short, engaging blog post about "{topic}" 
           based only on the following research data:
           {data} Return just the blog post content."""
    )
    
    chain = prompt | llm
    response = chain.invoke({"topic": topic, "data": data})
    
    print("Writing complete.")
    return {"blog_post": response.content}

def translator_node(state: AgentState):
    print("Translator is translating to Portuguese and French...")
    
    blog_post = state["blog_post"] if state["blog_post"] else ""
    
    llm = ChatOllama(model="llama3", temperature=0.7)
    
    prompt = ChatPromptTemplate.from_template(
        """You are a translator. 
           Translate the following text to Portuguese and French: "{blog_post}" 
           Return the blog post content in the 3 languages, the original in English plus the Portuguese and French version."""
    )
    
    chain = prompt | llm
    response = chain.invoke({"blog_post": blog_post})
    
    print("Translating complete.")
    return {"blog_post": response.content}

In [4]:
# Building the LangGraph
workflow = StateGraph(AgentState)

workflow.add_node("Researcher", researcher_node)
workflow.add_node("Writer", writer_node)
workflow.add_node("Translator", translator_node)

# Flow: Start -> Researcher -> Writer -> Translator -> END
workflow.set_entry_point("Researcher")
workflow.add_edge("Researcher", "Writer")
workflow.add_edge("Writer","Translator")
workflow.add_edge("Translator", END)

app = workflow.compile()

In [5]:
if __name__ == "__main__":
    print("Starting the Multi-Agent System...\n")
    
    inputs: AgentState = {
        "topic": "Electricity Markets in Alberta",
        "research_data": [],
        "blog_post": "",
    }
    
    result = app.invoke(inputs)
    
    print("\n---------------- FINAL OUTPUT ----------------\n")
    print(result["blog_post"])

Starting the Multi-Agent System...

Researcher is looking for latest news about: Electricity Markets in Alberta
Research complete.
Writer is writing the draft...
Writing complete.
Translator is translating to Portuguese and French...
Translating complete.

---------------- FINAL OUTPUT ----------------

Here are the translations:

**English (Original)**

**Rebooting Alberta's Electricity Markets: A Wake-Up Call for a Changing Landscape**

As we face a future dominated by rising demand and renewables, it's time to rethink the way we approach electricity markets in Alberta. The latest report from the International Energy Agency (IEA) serves as a stark reminder that our current systems are woefully inadequate.

In Alberta specifically, the need for an overhaul is clear. With the increasing reliance on solar power and other renewable energy sources, our traditional grid infrastructure is struggling to keep up. Moreover, the rising demand for electricity due to population growth and economi